In [1]:
import torch
# import scipy
# import umap
import random
# import itertools
import tsl
import numpy as np
# import pandas as pd
import networkx as nx
import torch_geometric
import lightning as L
import matplotlib.pyplot as plt

from tsl.datasets import PvUS
from tsl.data.datamodule import SpatioTemporalDataModule, TemporalSplitter
from tsl.data.preprocessing import StandardScaler

# from dataset.NCI1_dataset import NCI1
# from tqdm import trange
# from sklearn.model_selection import train_test_split

from models.DynGraphESN import DynGESNModel

from DMD.dmd import KANN

seed = 42
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

c:\Users\mgphy\Anaconda3\envs\koopman\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


for datasets:

https://github.com/dtortorella/dyngraphesn/tree/master

In [2]:
dataset = PvUS(root="/dataset")

In [4]:
print(dataset)

PvUS(length=52560, n_nodes=5016, n_channels=1)


In [3]:
print(f"Default similarity: {dataset.similarity_score}")
print(f"Available similarity options: {dataset.similarity_options}")
print("==========================================")

sim = dataset.get_similarity("distance")  # or dataset.compute_similarity()

# print("Similarity matrix W:")
# pd.DataFrame(sim)

Default similarity: distance
Available similarity options: {'distance', 'correntropy'}


In [3]:
connectivity = dataset.get_connectivity(threshold=0.1,
                                        include_self=False,
                                        normalize_axis=1,
                                        layout="edge_index")

In [4]:
edge_index, edge_weight = connectivity

print(f'edge_index {edge_index.shape}:\n', edge_index)
print(f'edge_weight {edge_weight.shape}:\n', edge_weight)

edge_index (2, 1090660):
 [[   0    0    0 ... 5015 5015 5015]
 [   2    3    6 ... 3025 3032 5009]]
edge_weight (1090660,):
 [0.00104493 0.00537338 0.00241709 ... 0.02544574 0.02086223 0.05187358]


In [5]:
torch_dataset = tsl.data.SpatioTemporalDataset(target=dataset.dataframe(),
                                      connectivity=connectivity,
                                      mask=dataset.mask,
                                      horizon=12,
                                      window=12,
                                      stride=1)
print(torch_dataset)

SpatioTemporalDataset(n_samples=52537, n_nodes=5016, n_channels=1)


In [6]:
sample = torch_dataset[0]
print(sample)

Data(
  input=(x=[t=12, n=5016, f=1], edge_index=[2, e=1090660], edge_weight=[e=1090660]),
  target=(y=[t=12, n=5016, f=1]),
  has_mask=False
)


In [7]:
input_size = sample.input.x.shape[2]

In [8]:
print(sample.pattern)

{'x': 't n f', 'edge_index': '2 e', 'edge_weight': 'e', 'y': 't n f'}


In [8]:
scalers = {'target': StandardScaler(axis=(0, 1))}

# Split data sequentially:
#   |------------ dataset -----------|
#   |--- train ---|- val -|-- test --|
splitter = TemporalSplitter(val_len=0.1, test_len=0.2)

dm = SpatioTemporalDataModule(
    dataset=torch_dataset,
    scalers=scalers,
    splitter=splitter,
    batch_size=64,
)

print(dm)

SpatioTemporalDataModule(train_len=None, val_len=None, test_len=None, scalers=[target], batch_size=64)


In [9]:
dm.setup()
print(dm)

SpatioTemporalDataModule(train_len=37815, val_len=4191, test_len=10507, scalers=[target], batch_size=64)


In [11]:
model = DynGESNModel(input_size=input_size,
                reservoir_size=100,
                input_scaling=1.,
                reservoir_layers=1,
                leaking_rate=0.9,
                spectral_radius=0.9,
                density=0.5,
                reservoir_activation='tanh',
                alpha_decay=False)

In [17]:
model(sample.input.x, sample.input.edge_index, None)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
class LitModel(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.model(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer